## Import Data

In [1]:
import numpy as np
import imdb 
import pandas as pd
from error_generator import Explicit_Missing_Value
from error_generator import Implicit_Missing_Value
from error_generator import White_Noise
from error_generator import Gaussian_Noise
from error_generator import Random_Active_Domain
from error_generator import Similar_Based_Active_Domain
from error_generator import Typo_Keyboard
from error_generator import Typo_Butterfingers
from error_generator import Word2vec_Nearest_Neighbor
from error_generator import Value_Selector
from error_generator import List_selected
from error_generator import Read_Write
from error_generator import Error_Generator
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn import linear_model
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score

anime_data = pd.read_csv("/Users/sezin/PycharmProjects/dsa-DC4ML/new_anime_data1.csv")

anime_data['episodes'] = anime_data['episodes'].replace('Unknown', np.nan)


/Users/sezin/.local/share/virtualenvs/dsa-DC4ML-_HMLejDF/lib/python3.7/site-packages/chainer-6.0.0a1-py3.7.egg/chainer/_environment_check.py:41: UserWarning: Accelerate has been detected as a NumPy backend library.
vecLib, which is a part of Accelerate, is known not to work correctly with Chainer.
We recommend using other BLAS libraries such as OpenBLAS.
For details of the issue, please see
https://docs.chainer.org/en/stable/tips.html#mnist-example-does-not-converge-in-cpu-mode-on-mac-os-x.

Also note that Chainer does not officially support Mac OS X.
Please use it at your own risk.

  ''')  # NOQA


## Cleaning

In [2]:
l1 = []
l2 = []
l3 = []


anime_data['genre'].fillna('',  inplace=True)
anime_data['overview'].fillna('',  inplace=True)
anime_data['type'].fillna('',  inplace=True)
for index, row in anime_data.iterrows():
    item = row['genre']
    if(pd.isnull(item)):
            item =""
    else:
        if isinstance(item, (list, tuple)):
            item = ','.join(item)
        else:
            item = item.replace(" ","")
            item = item.replace("[","")
            item = item.replace("]","")
            item = item.replace("'","")
    l1.append(item) 
      

for index, row in anime_data.iterrows():
    item = row['overview']
    if(pd.isnull(row['overview'])):
            item = ""
    else:
       # for item in anime_data['overview']:
            if isinstance(item, (list, tuple)):
                item = ','.join(item)
            else:
                item = item.replace("[","")
                item = item.replace("]","")
    l2.append(item) 

for index, row in anime_data.iterrows():
    item = row['type']
    if(pd.isnull(row['type'])):
        item = np.nan
    else:    
        if "movie" in item:
            item = "movie"
        else:
            item = "tv series"
    l3.append(item)   
    
    
anime_data['genre'] = l1   
anime_data['overview'] = l2
anime_data['type'] = l3

#drop dublicate
anime_data.drop_duplicates(inplace = True)



## Sampling

In [3]:
anime_train, anime_test = train_test_split(anime_data, test_size=0.2)

## Generating Errors

In [4]:
def error_generater(dataset, method, n, ignored_columns):

    myselector=List_selected()

    mygen=Error_Generator()

    new_dataset=mygen.error_generator(method_gen=method,selector=myselector,percentage=n,dataset=dataset,mute_column = ignored_columns)
    
    return new_dataset

In [5]:
# generate error in the training set
new_dataset = error_generater(anime_train.values.tolist(), Random_Active_Domain(),30, ignored_columns = [0,1,5,6,7])
Read_Write.write_csv_dataset("./{}.csv".format(Random_Active_Domain().name), new_dataset)
anime_train = pd.read_csv("./{}.csv".format(Random_Active_Domain().name), 
                            names = ['Unnamed: 0', 'anime_id', 'name','genre','type', 'episodes','rating', 'members', 'overview'])

---------Change according to random_active_domain method ---------------

row: 1462 col: 3 : 'Action,Kids' changed to 'SliceofLife,Supernatural'  
row: 1361 col: 4 : 'movie' changed to 'movie'  
row: 1436 col: 4 : 'tv series' changed to 'movie'  
row: 1120 col: 8 : ''A fantastical city portrayed in artful pictures that come to life through animation, music and background sounds.'' changed to '"When Natsumi's father dies she return to the forest which she had visited with him with his motorcycle. She meets and decides to stay with a family who also own a store nearby.::aghaemi"'  
row: 116 col: 4 : 'tv series' changed to 'tv series'  
row: 580 col: 8 : ''The adventures of young Osomatsu Matsuno and his five twin brothers.'' changed to ''10 children set out to get things changed which are threatening climate change for our planet. Presented by David De Rothschild''  
row: 1255 col: 3 : 'Hentai' changed to 'Adventure,Comedy,Fantasy'  
row: 1260 col: 3 : 'Drama' changed to 'Comedy,Talk-Sho

## Transformation

In [6]:
import nltk
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
def get_words(x):
    bagofwords=[]
    for i in x:
        if i[1]=='NN':
            bagofwords.append(i[0])
        elif i[1]=='NNS':
            bagofwords.append(i[0])
        elif i[1]=='NNP':
            bagofwords.append(i[0])
        elif i[1]=='NNPS':
            bagofwords.append(i[0])
        elif i[1]=='JJ':
            bagofwords.append(i[0])
        elif i[1]=='JJR':
            bagofwords.append(i[0])
        elif i[1]=='JJS':
            bagofwords.append(i[0])
        elif i[1]=='RB':
            bagofwords.append(i[0])
        elif i[1]=='RBR':
            bagofwords.append(i[0])
        elif i[1]=='RBS':
            bagofwords.append(i[0])
    return bagofwords

def clean_words(x):
    b=nltk.pos_tag(nltk.word_tokenize(x))
    result=get_words(b)
    return result

In [7]:
def feature_transformation(dataset):
    #Hot_Encoding
    df = dataset.genre.str.get_dummies(',')
    
    type_lb = LabelBinarizer()
    X = type_lb.fit_transform(dataset.type.values)

    dfOneHot = pd.DataFrame(X, columns = ["movie/TVseries" for i in range(X.shape[1])])
    dataset = pd.concat([dataset, dfOneHot], axis=1)
    dataset = pd.concat([dataset, df], axis=1)

    dataset['movie/TVseries'].fillna(0, inplace=True)
    
    #Bag of Words
    summary_doc = dataset['overview'].fillna("").map(clean_words)
    summary_doc =summary_doc.apply(','.join)
 
    vectorizer = TfidfVectorizer()
    overview_feature = vectorizer.fit_transform(summary_doc).toarray()

    df = pd.DataFrame(overview_feature, columns = ["word"+ str(int(i)) for i in range(overview_feature.shape[1])])
    dataset = pd.concat([dataset, df], axis=1)
    
    dataset = dataset.drop(columns=['Unnamed: 0', 'anime_id', 'name', 'genre', 'overview', 'type'])
    
    #drop NaN values
    dataset = dataset.dropna()

    return dataset



In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
anime_train = feature_transformation(anime_train)
anime_test = feature_transformation(anime_test)

In [9]:
anime_train.shape

(1550, 6888)

In [10]:
anime_test.shape

(137, 3717)

In [11]:
anime_train_columns = list(anime_train)
anime_test_columns = list(anime_test)

for col_name in anime_train_columns:
    if col_name not in anime_test_columns:
        anime_test[str(col_name)] = 0
        
for col_name in anime_test_columns:
    if col_name not in anime_train_columns:
        anime_test.drop([str(col_name)], axis=1, inplace=True)
        

In [12]:
#anime_train = anime_train.dropna()
print(anime_train.shape)
print(anime_test.shape)

(1550, 6888)
(137, 6888)


In [13]:
anime_y_train = anime_train['rating']
anime_X_train = anime_train.drop(columns=['rating'])

anime_y_test = anime_test['rating']
anime_X_test = anime_test.drop(columns=['rating'])


## Feature Selection

In [14]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

selector = SelectKBest(score_func=f_regression,k=700)#anime_X_test.shape[1]-1)
features = selector.fit(anime_X_train, anime_y_train)

# summarize scores
np.set_printoptions(precision=3)
#print(fit.scores_)

print(anime_X_train.shape)
print(anime_X_test.shape)
anime_X_train = features.transform(anime_X_train)
anime_X_test = features.transform(anime_X_test)
print(anime_X_train.shape)
print(anime_X_test.shape)

(1550, 6887)
(137, 6887)
(1550, 700)
(137, 700)


/Users/sezin/.local/share/virtualenvs/dsa-DC4ML-_HMLejDF/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
/Users/sezin/.local/share/virtualenvs/dsa-DC4ML-_HMLejDF/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/sezin/.local/share/virtualenvs/dsa-DC4ML-_HMLejDF/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/sezin/.local/share/virtualenvs/dsa-DC4ML-_HMLejDF/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


## Training - Testing - Evaluation

In [15]:
def training(model, dataset, label):
    clf = model
    clf.fit(dataset, label)
    return clf

def testing_evaluation(model, testset):
    # Make predictions using the testing set
    anime_y_pred = model.predict(testset)

    # The mean absolute error
    print("Mean absolute error: %.2f" % np.sqrt(mean_absolute_error(anime_y_test, anime_y_pred)))

    # The mean squared error
    print("Mean squared error: %.2f" % np.sqrt(mean_squared_error(anime_y_test, anime_y_pred)))

    # Explained variance score: 1 is perfect prediction
    print('Variance score: %.2f' % r2_score(anime_y_test, anime_y_pred))

### Linear Regression

In [16]:
linear = training(model = linear_model.LinearRegression(), dataset = anime_X_train, label= anime_y_train)
testing_evaluation(linear, anime_X_test)

/Users/sezin/.local/share/virtualenvs/dsa-DC4ML-_HMLejDF/lib/python3.7/site-packages/sklearn/linear_model/base.py:485: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)


Mean absolute error: 0.92
Mean squared error: 1.10
Variance score: -0.30


### Lesso Regression

In [17]:
clf = training(model = linear_model.Lasso(alpha=0.1), dataset = anime_X_train, label= anime_y_train)
testing_evaluation(clf, anime_X_test)
anime_y_pred_lesso = clf.predict(anime_X_test)

Mean absolute error: 0.82
Mean squared error: 0.86
Variance score: 0.20


# Visualisation

In [18]:
import matplotlib as mpl
mpl.use('TkAgg')
import matplotlib.pyplot as plt

In [ ]:
anime_y_test = anime_y_test.reset_index()
print(type(anime_y_test))

<class 'pandas.core.frame.DataFrame'>


In [ ]:
anime_y_test = anime_y_test['rating']

In [ ]:
anime_y_pred_lesso = clf.predict(anime_X_test)
anime_y_pred_linear = linear.predict(anime_X_test)

outputDataFrame = pd.DataFrame(anime_y_test)
outputDataFrame.reset_index(inplace=True)
outputDataFrame = outputDataFrame
outputDataFrame['predicted lesso'] = anime_y_pred_lesso
outputDataFrame.drop(columns='index')
outputDataFrame.to_csv('/Users/sezin/PycharmProjects/dsa-DC4ML/obj/prediction_with_errors.csv')
print(list(outputDataFrame))

['index', 'rating', 'predicted lesso']


In [ ]:
import matplotlib as mpl
mpl.use('TkAgg')
import matplotlib.pyplot as plt

def error_lower_than_p(anime_y_pred, anime_y_test):
    bins = [float(i)/100 for i in range(0, 100)]
    relative_error = []
    
    for j in range(0, len(anime_y_test)):
        relative_error.append(abs((anime_y_test[j]-anime_y_pred[j])/anime_y_test[j]))

    error_lower_than_p = []
    for k in range(0, len(bins)):
        count = 0
        for l in range(0, len(relative_error)):
            if relative_error[l] < bins[k]:
                count += 1
        error_lower_than_p.append(count/len(relative_error))
    return(error_lower_than_p)

outputDataFrame_withoutErrors = pd.read_csv('/Users/sezin/PycharmProjects/dsa-DC4ML/obj/prediction_without_errors.csv')
outputDataFrame_withErrors = pd.read_csv('/Users/sezin/PycharmProjects/dsa-DC4ML/obj/prediction_with_errors.csv')

anime_y_pred = clf.predict(anime_X_test)

error_lower_than_p_lesso_without_errors = error_lower_than_p(outputDataFrame_withoutErrors['predicted lesso'], outputDataFrame_withoutErrors['rating'])
error_lower_than_p_lesso_with_errors = error_lower_than_p(outputDataFrame_withErrors['predicted lesso'], outputDataFrame_withErrors['rating'])

bins = [float(i)/100 for i in range(0, 100)]

plt.plot(bins, error_lower_than_p_lesso_without_errors)
plt.plot(bins, error_lower_than_p_lesso_with_errors)
plt.title(label='Prediction with Errors vs. without Errors')
plt.xlabel("relative error p")
plt.ylabel("likelihood of prediction to be better than p")
plt.legend(['without Errors', 'with Errors'], loc='lower right')
plt.show()